In [1]:
import cv2
from ultralytics import YOLOWorld
import supervision as sv
import os

In [2]:
# Путь к папке с изображениями
PATH_TO_IMAGES = "sample_substraction"
# Префикс директории, куда сбрасываются аннотированные изображения (для визуальной проверки результатов)
OUTPUT_DIR_PREFIX = "annotated"
# Размер батча
BATCH_SIZE = 5
# Порог для NMS
NMS_THRESHOLD = 1e-9
# Порог уверенности распознавания класса
CONFIDENCE = 0.45
# Размер изображения
image_size = (640, 640)
# Список картинок
list_of_images = os.listdir(PATH_TO_IMAGES)
# Промпт к классу для модели
license_platee_prompt = """
russian car license plate: 
- white rectangular metal plate
- black cyrillic characters
- format: 1 letter, 3 digits, 2 letters, regional code
- example: A123BC777
- not a car manufacturer badge or model emblem
- not a part of vehicle body (door, bumper, headlight, etc.)
"""
# Словарь классов и промптов к ним
classes_dict = {"license_plate": license_platee_prompt}

In [3]:
def check_output_dir_availability(dir_name):
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)
check_output_dir_availability(f"{OUTPUT_DIR_PREFIX}_{PATH_TO_IMAGES}")

In [4]:
def perfom_annotation_of_images(path_to_images, 
                                output_dir_prefix, 
                                batch_size, 
                                confidence, 
                                image_size, 
                                nms_threshold, 
                                list_of_images, 
                                classes_dict):
    # Инициализируем модель
    model = YOLOWorld("yolov8x-worldv2.pt")

    list_of_images_with_relative_path = [os.path.join(path_to_images, image) for image in list_of_images]

    classes = []

    # Запихиваем промпт в список классов
    for key in classes_dict:
        classes.append(classes_dict[key])

    classes.append("")

    model.set_classes(classes)

    # Создаем датасет
    dataset = sv.DetectionDataset(classes=list(classes_dict.keys()), 
                                images=list_of_images_with_relative_path, 
                                annotations={list_of_images_with_relative_path[i]: None 
                                            for i in range(len(list_of_images_with_relative_path))})

    # Итерация по списку картинок: обрабатываем картинки батчами
    for i in range(0, len(list_of_images_with_relative_path), batch_size):
        batch = list_of_images_with_relative_path[i:batch_size + i + 1]
        lst_of_cv2_instances = [cv2.imread(image) for image in batch]

        # Запускаем детекцию (подаем батч картинок) с порогом уверенности 
        # TODO если классов несколько - для каждого из них должен быть разный порог уверенности
        results = model.predict(batch, conf=confidence, imgsz=image_size, iou = nms_threshold)

        # Итерация по каждой детекции в батче
        for index in range(len(results)):
            # Получение детекций
            detections = sv.Detections.from_ultralytics(results[index])

            # Инициализация экземпляров классов для боксов и подписей
            annotator = sv.BoxAnnotator()
            label_annotator = sv.LabelAnnotator()

            # вывод доп информации (уверенность)
            labels = [
                f"{confidence:.2f}"
                for confidence
                in detections.confidence
            ]

            # Непосредственно, рисуем боксы и подписи
            image_with_bounding_boxes = annotator.annotate(scene = lst_of_cv2_instances[index], 
                                                detections = detections)
            annotated_image = label_annotator.annotate(scene = image_with_bounding_boxes, 
                                                    detections = detections, 
                                                    labels=labels)
            
            # Сохраняем размеченную картинку
            cv2.imwrite(f"{output_dir_prefix}_{batch[index]}", annotated_image)

            # Вносим в датасет инфу по детекциям для конкретной картинки
            dataset.annotations[batch[index]] = detections
    return dataset
        
detection_data = perfom_annotation_of_images(PATH_TO_IMAGES, 
                                             OUTPUT_DIR_PREFIX, 
                                             BATCH_SIZE, 
                                             CONFIDENCE, 
                                             image_size, 
                                             NMS_THRESHOLD, 
                                             list_of_images, 
                                             classes_dict)


0: 640x640 1 
russian car license plate: 
- white rectangular metal plate
- black cyrillic characters
- format: 1 letter, 3 digits, 2 letters, regional code
- example: A123BC777
- not a car manufacturer badge or model emblem
- not a part of vehicle body (door, bumper, headlight, etc.)
, 47.5ms
1: 640x640 1 
russian car license plate: 
- white rectangular metal plate
- black cyrillic characters
- format: 1 letter, 3 digits, 2 letters, regional code
- example: A123BC777
- not a car manufacturer badge or model emblem
- not a part of vehicle body (door, bumper, headlight, etc.)
, 47.5ms
2: 640x640 2 
russian car license plate: 
- white rectangular metal plate
- black cyrillic characters
- format: 1 letter, 3 digits, 2 letters, regional code
- example: A123BC777
- not a car manufacturer badge or model emblem
- not a part of vehicle body (door, bumper, headlight, etc.)
s, 47.5ms
3: 640x640 1 
russian car license plate: 
- white rectangular metal plate
- black cyrillic characters
- format: 1

In [5]:
def create_dataset_in_yolo_format(dataset):
    # Разбиваем датасет на train, test и val
    train_dataset, test_dataset = dataset.split(split_ratio=0.8, random_state=42, shuffle=True)
    # Дополнительно разобьем test, чтобы получить val
    test_dataset, val_dataset = test_dataset.split(split_ratio=0.5, random_state=42)
    dct_of_datasets = {'train': train_dataset,
                   'test': test_dataset,
                   'val': val_dataset}
    for dataset_type, dataset_instance in dct_of_datasets.items():
        # Экспортируем в YOLO-формат (data_yaml генерируется невалидный для CVAT, нужно создавать самому)
        dataset_instance.as_yolo(
        images_directory_path=f"license_plate_detection_dataset_v2/{dataset_type}/images",
        annotations_directory_path=f"license_plate_detection_dataset_v2/{dataset_type}/labels",
        )
        
create_dataset_in_yolo_format(detection_data)